## Geo-parsing and Geo-Visualization of Road Traffic Crash Incident Locations from Print Media for Emergency Response and Planning

### Code for RTC-NER Model

#### Setup the Environment

In [4]:
!python3 -m pip install tensorflow
!pip install spacy-transformers seqeval folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 25.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver

In [5]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.7.4                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.58+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.7.1)        



In [6]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

from seqeval.metrics import classification_report #to evaluate model
from spacy.tokens import DocBin #to load model

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


### Load the annotated Training and Test datasets

In [ ]:
rtc_train = open('rtc_train_annotations.json')
TRAIN_DATA = json.load(rtc_train)

rtc_test = open('rtc_test_annotations.json')
TEST_DATA = json.load(rtc_test)

#### Convert the .json files into .spacy format

In [ ]:
# Create DocBin Objects
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

for text, annot in tqdm(TEST_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./test_data.spacy") # save the docbin object

100%|██████████| 1/1 [00:00<00:00, 52.08it/s]


### Train the RTC-NER Model

In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

2023-12-07 21:31:18.361856: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 21:31:18.361916: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 21:31:18.362000: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 21:31:19.515805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data 

In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-12-07 21:31:28.731760: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 21:31:28.731826: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 21:31:28.731882: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 21:31:29.873439: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline

## **Evaluation of the RTC-NER model**

In [7]:
!python -m spacy evaluate model-best/ "./test_data.spacy"

ℹ Using CPU
/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.6.1 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

================================== Results ==================

### Toponym Recognition Using the RTC-NER Model

In [12]:
#Load the newly created model
rtc_ner = spacy.load("model-best")

In [13]:
doc1 = rtc_ner(""" No fewer than five people were confirmed dead on Sunday while 12 others sustained varying degrees of injuries in an accident near the Foursquare Camp in Ajebo on the Lagos-Ibadan Expressway.

Mrs Florence Okpe, the Public Education Officer of the Federal Road Safety Corps (FRSC) in Ogun, confirmed the development in an interview with newsmen in Abeokuta.

She stated that the injured people were taken to Victory Hospital, Ogbere for medical attention while the dead were deposited at a morgue in Ipara community, near Abeokuta.

""")

In [14]:
spacy.displacy.render(doc1, style="ent", jupyter=True) # display in Jupyter

In [15]:
for ent in doc1.ents:
	print(ent.text, " ->>>> ", ent.label_)

five people  ->>>>  CASUALTY
12 others  ->>>>  INJURED
near the Foursquare Camp  ->>>>  LANDMARK
Ajebo  ->>>>  TOWN
Lagos-Ibadan Expressway.  ->>>>  ROAD
Victory Hospital, Ogbere  ->>>>  HOSPITAL


### Toponym Resolution (Geocoding)

In [18]:
#Displaying selected RTC location information
locations = []
location_tag = []

tags = ['ROAD', 'LANDMARK', 'TOWN', 'LGA', 'STATE', 'HOSPITAL']

for tag in tags:
  for ent in doc1.ents:
      if ent.label_ == tag:
        locations.append(ent.text)
        #location_tag.append(tag)

#print(locations)
#print(location_tag)

#function to concatenate all elements of locations list (except the last element) as rtc_site
def conexclast(strlst):
    output = ""
    for elem in strlst:
       strng = str(elem)
       output = output+strng
    return ', '.join(strlst[0:-1])


rtc_site = conexclast(locations[1:])
rtc_road = locations[0]
hospital = locations[-1]


rtc_places = []
rtc_places.append(rtc_road)
rtc_places.append(rtc_site)
rtc_places.append(hospital)

print("Road Traffic Crash Location details:", rtc_places)

Road Traffic Crash Location details: ['Lagos-Ibadan Expressway.', 'near the Foursquare Camp, Ajebo', 'Victory Hospital, Ogbere']


In [19]:
#Using Google maps geocoding api
import requests
import pandas as pd

API_KEY = 'xxxxxxxx' #insert your Google Geocoding API key inplace of xxxxxxxx

df = pd.DataFrame()

latitude_list = []
longitude_list = []

for loc in rtc_places:
  params = {
    'key': API_KEY,
    'address': loc
  }

  base_url = 'https://maps.googleapis.com/maps/api/geocode/json'
  response = requests.get(base_url, params=params).json()

  geometry = response['results'][0]['geometry']
  lat = geometry['location']['lat']
  lon = geometry['location']['lng']
  latitude_list.append(lat)
  longitude_list.append(lon)

df = pd.DataFrame({'Location': rtc_places})

df['Latitude'] = latitude_list
df['Longitude'] = longitude_list

df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

df



,Location,Latitude,Longitude
0,Lagos-Ibadan Expressway.,6.923588,3.636422
1,"near the Foursquare Camp, Ajebo",7.109120,3.723304
2,"Victory Hospital, Ogbere",6.739754,4.164174


### Geo-Visualization

In [20]:
import folium

In [21]:
df

,Location,Latitude,Longitude
0,Lagos-Ibadan Expressway.,6.923588,3.636422
1,"near the Foursquare Camp, Ajebo",7.109120,3.723304
2,"Victory Hospital, Ogbere",6.739754,4.164174


In [23]:
#coordinate of the road
road_coord = [df['Latitude'].iloc[0], df['Longitude'].iloc[0]]
print(road_coord)

[6.9235877, 3.6364221]


In [24]:
map = folium.Map(location=road_coord, zoom_start=13)

folium.Marker(road_coord, popup='RTC Road').add_to(map)
map

In [25]:
# Coordinates of the points
points = []
for i in range(df.shape[0]):
    rtc_coords = [df['Latitude'].iloc[i], df['Longitude'].iloc[i]]
    points.append(rtc_coords)

points

[[6.9235877, 3.6364221],
 [7.1091204, 3.7233044],
 [6.7397541, 4.164174099999999]]

In [26]:
for point in points:
    folium.Marker(point, popup='Point').add_to(map)
map